# mnist数据集，三层神经网络
 
 正确率不高
 
 blog：https://indevsync.azurewebsites.net/proxy.php?https://missinglink.ai/guides/tensorflow/tensorflow-conv2d-layers-practical-guide/
 
 出自 https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network_raw.py


## 初始化，使用tf.v1，关掉debug信息

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


## 设置超参数

In [2]:
learning_rate = 0.001
num_steps = 50
batch_size = 100
display_step = 10

num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)


## 加载mnist数据

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


x_train = np.reshape(x_train,(-1,784))
x_test = np.reshape(x_test,(-1,784))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


## 设置训练batch generator

In [4]:
def train_generator(batch_size):
    for i in range(0, x_train.shape[0], batch_size):
        yield x_train[i:i+batch_size,:], y_train[i:i+batch_size]
        
def test_generator(batch_size):
    for i in range(0, x_test.shape[0], batch_size):
        yield x_test[i:i+batch_size,:], y_test[i:i+batch_size]


## 输入变量Placeholder

In [5]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)


# 神经网络部分
## 卷积层

In [6]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')



## 核心函数

In [7]:
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


## 存储各个变量

In [8]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}


# 训练和评估
## 各个评估变量

In [9]:
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## 训练部分

In [10]:
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next(train_generator(batch_size))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

            print("Testing Accuracy:", \
                    sess.run(accuracy, feed_dict={X: x_test[:batch_size,:],
                        Y: y_test[:batch_size],
                                              keep_prob: 1.0}))
    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: x_test,
                Y: y_test,
                                      keep_prob: 1.0}))


Step 1, Minibatch Loss= 49788.4414, Training Accuracy= 0.100
Testing Accuracy: 0.11
Step 10, Minibatch Loss= 10803.1553, Training Accuracy= 0.530
Testing Accuracy: 0.42
Step 20, Minibatch Loss= 2762.2957, Training Accuracy= 0.790
Testing Accuracy: 0.59
Step 30, Minibatch Loss= 555.9905, Training Accuracy= 0.940
Testing Accuracy: 0.65
Step 40, Minibatch Loss= 429.2147, Training Accuracy= 0.960
Testing Accuracy: 0.71
Step 50, Minibatch Loss= 306.5718, Training Accuracy= 0.980
Testing Accuracy: 0.72
Optimization Finished!
Testing Accuracy: 0.6506
